In [1]:
import pandas as pd
import os
import sqlite3

path = r"C:\Users\HP\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2"


In [2]:
customers = pd.read_csv(os.path.join(path, "olist_customers_dataset.csv"))


In [3]:
customers.head()
customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [4]:
conn = sqlite3.connect("olist_ecommerce.db")


In [5]:
customers.to_sql("customers", conn, if_exists="replace", index=False)


99441

In [6]:
orders = pd.read_csv(os.path.join(path, "olist_orders_dataset.csv"))
orders.to_sql("orders", conn, if_exists="replace", index=False)


99441

In [7]:
order_items = pd.read_csv(os.path.join(path, "olist_order_items_dataset.csv"))
order_items.to_sql("order_items", conn, if_exists="replace", index=False)


112650

In [8]:
products = pd.read_csv(os.path.join(path, "olist_products_dataset.csv"))
products.to_sql("products", conn, if_exists="replace", index=False)


32951

In [9]:
payments = pd.read_csv(os.path.join(path, "olist_order_payments_dataset.csv"))
payments.to_sql("payments", conn, if_exists="replace", index=False)


103886

In [10]:
reviews = pd.read_csv(os.path.join(path, "olist_order_reviews_dataset.csv"))
reviews.to_sql("reviews", conn, if_exists="replace", index=False)


99224

In [11]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name
0,customers
1,orders
2,order_items
3,products
4,payments
5,reviews


In [13]:
query = """
SELECT
  strftime('%Y-%m', o.order_purchase_timestamp) AS month,
  ROUND(SUM(oi.price + oi.freight_value), 2) AS total_revenue
FROM orders o
JOIN order_items oi
  ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY month
ORDER BY month;
"""
pd.read_sql(query, conn)


,month,total_revenue
0,2016-09,143.46
1,2016-10,46490.66
2,2016-12,19.62
3,2017-01,127482.37
4,2017-02,271239.32
5,2017-03,414330.95
6,2017-04,390812.40
7,2017-05,566851.40
8,2017-06,490050.37
9,2017-07,566299.08


### Insight
Revenue grew rapidly from early 2017, peaking in November 2017, and remained consistently high through 2018, indicating strong business scaling and seasonal demand patterns.

### Business Action
Prioritize marketing campaigns and inventory planning during peak months (especially Q4) and analyze mid-2018 performance drivers to maintain long-term growth.


In [14]:
query = """
SELECT
  p.product_category_name,
  ROUND(SUM(oi.price), 2) AS category_revenue
FROM order_items oi
JOIN products p
  ON oi.product_id = p.product_id
JOIN orders o
  ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY p.product_category_name
ORDER BY category_revenue DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,product_category_name,category_revenue
0,beleza_saude,1233131.72
1,relogios_presentes,1166176.98
2,cama_mesa_banho,1023434.76
3,esporte_lazer,954852.55
4,informatica_acessorios,888724.61
5,moveis_decoracao,711927.69
6,utilidades_domesticas,615628.69
7,cool_stuff,610204.10
8,automotivo,578966.65
9,brinquedos,471286.48


### Insight
Revenue is concentrated in a small set of categories, with Beauty & Health, Watches & Gifts, and Home & Lifestyle products contributing the largest share of total revenue, highlighting strong demand in these segments.

### Business Action
Focus inventory planning and marketing investments on top-performing categories, and leverage them for cross-selling and promotional strategies to improve overall revenue and average order value.


In [15]:
query = """
SELECT
  CASE
    WHEN order_count = 1 THEN 'One-time Customer'
    ELSE 'Repeat Customer'
  END AS customer_type,
  COUNT(*) AS customer_count
FROM (
  SELECT
    customer_id,
    COUNT(order_id) AS order_count
  FROM orders
  WHERE order_status = 'delivered'
  GROUP BY customer_id
)
GROUP BY customer_type;
"""
pd.read_sql(query, conn)


,customer_type,customer_count
0,One-time Customer,96478


### Insight
The majority of customers are one-time buyers, indicating low customer retention despite strong customer acquisition. This highlights a gap in long-term customer engagement and repeat purchasing behavior.

### Business Action
Introduce retention-focused initiatives such as loyalty programs, personalized recommendations, and post-purchase engagement campaigns to increase repeat purchases and improve customer lifetime value.


In [16]:
query = """
SELECT
  CASE
    WHEN julianday(o.order_delivered_customer_date)
         - julianday(o.order_estimated_delivery_date) <= 0
    THEN 'On Time'
    ELSE 'Late'
  END AS delivery_status,
  ROUND(AVG(r.review_score), 2) AS avg_review_score
FROM orders o
JOIN reviews r
  ON o.order_id = r.order_id
WHERE o.order_delivered_customer_date IS NOT NULL
GROUP BY delivery_status;
"""
pd.read_sql(query, conn)


,delivery_status,avg_review_score
0,Late,2.57
1,On Time,4.29


### Insight
Delivery delays significantly reduce customer satisfaction. Orders delivered late receive an average review score of 2.57, compared to 4.29 for on-time deliveries, highlighting delivery performance as a critical driver of customer experience.

### Business Action
Focus on reducing late deliveries by improving logistics planning, setting accurate delivery expectations, and introducing proactive customer communication or compensation strategies for delayed orders.


In [17]:
query = """
SELECT
  o.customer_id,
  ROUND(SUM(oi.price + oi.freight_value), 2) AS lifetime_value
FROM orders o
JOIN order_items oi
  ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY o.customer_id
ORDER BY lifetime_value DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,customer_id,lifetime_value
0,1617b1357756262bfa56ab541c47bc16,13664.08
1,ec5b2ba62e574342386871631fafd3fc,7274.88
2,c6e2731c5b391845f6800c97401a43a9,6929.31
3,f48d464a0baaea338cb25f816991ab1f,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66
5,05455dfa7cd02f13d132aa7a6a9729c6,6081.54
6,df55c14d1476a9a3467f131269c2477f,4950.34
7,24bbf5fd2f2e1b359ee7de94defc4a15,4764.34
8,3d979689f636322c62418b6346b1c6d2,4681.78
9,1afc82cd60e303ef09b4ef9837c9505c,4513.32


### Insight
A small subset of customers generates disproportionately high lifetime value, highlighting the importance of high-value (VIP) customers in overall revenue contribution.

### Business Action
Implement targeted retention and personalization strategies for high-value customers, such as exclusive rewards and tailored offers, and use their behavior patterns to inform look-alike customer targeting.
